In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from auc_tools import standard_auc, monte_carlo_auc

In [ ]:
#1. Visualize the zoo of response curves

import numpy as np
import matplotlib.pyplot as plt

# Create time points
x = np.linspace(0, 120, 500)

# Create a figure for plotting
plt.figure(figsize=(12, 16))

# Generate and plot different response curves
curve_types = ['skewed_gaussian', 'biexponential', 'gamma', 'lognormal', 
               'weibull', 'bateman', 'inverse_gaussian']

sampled_curves = sample_curves(parameter_bounds)


for i, curve_type in enumerate(curve_types):

    plt.subplot(len(curve_types), 1, i+1)
    plt.title(f"{curve_type.replace('_', ' ').capitalize()}")

    # Sample random parameters within bounds
    bounds = parameter_bounds[curve_type]
    params = {}
    for param, (min_val, max_val) in bounds.items():
        params[param] = np.random.uniform(min_val, max_val)
    
    for params in sampled_curves[curve_type]:
        response, start, end = generate_response_curve(x, curve_type=curve_type, params=params)

        # Plot
        plt.plot(x, response)
        plt.grid(True, alpha=0.3)
        ax = plt.gca()
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)

plt.xlabel('Time (min)', fontsize=12)
plt.gcf().text(-.01, 0.5, 'Density', va='center', rotation='vertical', fontsize=12)

plt.tight_layout()
plt.savefig('../results/response_curves_zoo.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Generate actual response curves using the sampled parameters
response_curves_set = []

ctr = 0
for num_participants in range(5, 42, 5):  # Participants from 5 to 41 with step size of 5
    for num_datapoints in range(4, 11):  # Datapoints from 4 to 10
        for curve_type, param_list in sampled_curves.items():
            for index, params in enumerate(param_list):
                # Generate the true response curve
                response, support_start, support_end = generate_response_curve(
                    x, curve_type=curve_type, params=params, threshold=0.01
                )
                
                # Define gridpoints within the active support
                gridpoints = np.linspace(support_start, support_end, num_datapoints)
                #gridpoints = find_representative_points(x, response, num_datapoints)
                
                # Evaluate the response at the gridpoints
                true_values = np.interp(gridpoints, x, response)
                
                # Generate random values around the true values with 20% standard deviation
                random_values = np.random.normal(
                    loc=true_values, scale=true_values * 0.2, size=(num_participants, len(gridpoints))
                )
                
                # Calculate mean and standard deviation of the responses
                mean_responses = random_values.mean(axis=0)
                std_responses = random_values.std(axis=0)
                
                # Calculate the true AUC over the support
                true_auc = np.trapz(response[(x >= support_start) & (x <= support_end)], x[(x >= support_start) & (x <= support_end)])


                # Append the result to the response_curves_set
                response_curves_set.append({
                    'index': index,
                    'curve_type': curve_type,
                    'parameters': params,
                    'x_vals': gridpoints,
                    'num_participants': num_participants,
                    'mean_responses': mean_responses,
                    'std_responses': std_responses,
                    'support_start': support_start,
                    'support_end': support_end,
                    'true_auc': true_auc,
                })

                if ctr % 100 == 0:
                    print(f"Generated {ctr} of 3920 curves ")
                ctr += 1


In [ ]:
#This part we should be able to run in a loop, sweeping over number of participants and datapoints

results = {
    "grid_points_in_graph": [],
    "participants": [],
    "curve_type": [],
    "parameters": [],
    "standard_auc_mean": [],
    "standard_auc_lower_bound": [],
    "standard_auc_upper_bound": [],
    "mc_auc_mean": [],
    "mc_auc_std": [],
    "bayesian_auc_mean": [],     # New
    "bayesian_hdi_low": [],      # New
    "bayesian_hdi_high": [],     # New
    "bayesian_iterations": [],    # New
    "current_grid": [],
    "means": [],
    "stds": [],
    "true_auc": []
}

for ctr, curve in enumerate(response_curves_set):
    current_grid, means, stds = curve['x_vals'], curve['mean_responses'], curve['std_responses']

    # Calculate standard AUC
    standard_auc_mean, auc_lower_bound, auc_upper_bound = standard_auc(
        current_grid, means, stds, curve['num_participants']
    )

    # Calculate Monte Carlo AUC
    mc_auc_mean, mc_auc_std = monte_carlo_auc(current_grid, means, stds, n_sim=1000)
    
    # Calculate Bayesian AUC
    posterior_samples, bayes_mean, bayes_hdi_low, bayes_hdi_high, bayes_iterations = bayesian_auc(
        current_grid, means, stds, n_iterations=1000, convergence_threshold=0.000001
    )

    # Store all results
    results["standard_auc_mean"].append(standard_auc_mean)
    results["grid_points_in_graph"].append(len(current_grid))
    results["participants"].append(curve['num_participants'])
    results["curve_type"].append(curve['curve_type'])
    results["parameters"].append(curve['parameters'])       
    results["standard_auc_lower_bound"].append(auc_lower_bound)
    results["standard_auc_upper_bound"].append(auc_upper_bound)
    results["mc_auc_mean"].append(mc_auc_mean)
    results["mc_auc_std"].append(mc_auc_std)
    results["bayesian_auc_mean"].append(bayes_mean)        # New
    results["bayesian_hdi_low"].append(bayes_hdi_low)      # New
    results["bayesian_hdi_high"].append(bayes_hdi_high)    # New
    results["bayesian_iterations"].append(bayes_iterations) # New
    results["current_grid"].append(current_grid)
    results["means"].append(means)
    results["stds"].append(stds)
    results["true_auc"].append(curve['true_auc'])
    
    if ctr % 100 == 0:
        print(f"Processed {ctr} curves out of {len(response_curves_set)}")

# Create DataFrame and add error metrics
df = pd.DataFrame(results) 

# Add relative error and coverage for all methods
df["mc_relative_error"] = abs(df["mc_auc_mean"] - df["true_auc"]) / df["true_auc"]
df["standard_relative_error"] = abs(df["standard_auc_mean"] - df["true_auc"]) / df["true_auc"]
df["bayesian_relative_error"] = abs(df["bayesian_auc_mean"] - df["true_auc"]) / df["true_auc"]

# Calculate standard deviations for coverage calculation
df['standard_auc_simple_std'] = (df['standard_auc_mean'] - df['standard_auc_lower_bound'] + 
                                df['standard_auc_upper_bound'] - df['standard_auc_mean']) / 2
df['bayesian_auc_std'] = (df['bayesian_hdi_high'] - df['bayesian_hdi_low']) / 3.29  # 89% HDI

# Add coverage indicators for all methods
df["mc_coverage"] = (
    (df["mc_auc_mean"] - 1.96 * df["mc_auc_std"] <= df["true_auc"]) &
    (df["mc_auc_mean"] + 1.96 * df["mc_auc_std"] >= df["true_auc"])
).astype(int)

df["standard_coverage"] = (
    (df["standard_auc_mean"] - 1.96 * df["standard_auc_simple_std"] <= df["true_auc"]) &
    (df["standard_auc_mean"] + 1.96 * df["standard_auc_simple_std"] >= df["true_auc"])
).astype(int)

df["bayesian_coverage"] = (
    (df["bayesian_hdi_low"] <= df["true_auc"]) &
    (df["bayesian_hdi_high"] >= df["true_auc"])
).astype(int)




In [ ]:
df.to_csv('../results/auc_results.csv', index=False)

In [ ]:
from auc_tools import compare_methods_performance
import pandas as pd

# Create a DataFrame to store all results
all_results = []

# Analyze each curve type separately
for curve_type in df['curve_type'].unique():
    curve_data = df[df['curve_type'] == curve_type]
    
    # Create dictionary of methods with their estimates and standard errors
    methods = {
        'Standard': (
            curve_data['standard_auc_mean'].values,
            curve_data['standard_auc_simple_std'].values
        ),
        'Monte Carlo': (
            curve_data['mc_auc_mean'].values,
            curve_data['mc_auc_std'].values
        ),
        'Bayesian': (
            curve_data['bayesian_auc_mean'].values,
            curve_data['bayesian_auc_std'].values
        )
    }
    
    # Calculate performance metrics
    results = compare_methods_performance(curve_data['true_auc'].values, methods)
    
    # Add curve type as index name
    results.index.name = 'Method'
    results = results.reset_index()
    results.insert(0, 'Curve Type', curve_type)
    
    all_results.append(results)

# Combine all results
final_results = pd.concat(all_results, ignore_index=True)

# Display results in a nicely formatted table
print("Performance Metrics by Curve Type and Method")
print("(Values shown as: Estimate (Monte Carlo Standard Error))")
print("\nBias, Empirical SE, RMSE, and Coverage (95% CI)")
print("="*80)
print(final_results.to_string(index=False))

# Save results to CSV
final_results.to_csv('performance_metrics.csv', index=False)

# Create separate tables for each metric for easier reporting
metrics = ['bias', 'empirical_se', 'rmse', 'coverage']
for metric in metrics:
    print(f"\n{metric.upper()} by Curve Type and Method:")
    print("="*50)
    pivot_table = final_results.pivot(
        index='Curve Type',
        columns='Method',
        values=metric
    )
    print(pivot_table.to_string())
    # Save individual metric tables
    pivot_table.to_csv(f'../results/{metric}_by_method.csv')